# Vacations Analysis

Martín Singla - Data Analyst 

### 0.0 - Project Description (Part 2)

In this second part of the project we utilize the cities dataframe from Part 1, which contains meteorological conditions for over 2K cities around the world. We plot Humidity (%) by city using Gmap module. Following that, we filter cities based on our optimal weather conditions (low humidity, chill temperature, etc.) and narrow down the df to 10~20 cities. Finally, we use Google Places API to find hotels in a 5km radius from those selected cities, and plot them using Gmap. 

In [133]:
#Load libraries
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import requests
import gmaps

#API keys
from api_keys import gkey
gmaps.configure(api_key=gkey)

#import data
dat = pd.read_csv("results/global_city_weather_df.csv")
dat.head()

,City,Country_code,Lat,Lng,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Preassure,Sea Level (ft)
0,los llanos de aridane,es,28.6585,-17.9182,60.66,68.0,20.0,5.75,1014.0,NaN
1,paamiut,gl,61.9940,-49.6678,36.79,84.0,3.0,5.91,1025.0,1025.0
2,mataura,pf,-46.1927,168.8643,53.01,82.0,100.0,11.01,1025.0,NaN
3,arman,ru,59.7000,150.1667,34.18,74.0,66.0,3.56,1023.0,1023.0
4,hilo,us,19.7297,-155.0900,78.04,51.0,40.0,9.22,1018.0,NaN


##### Humidity Heatmap

In [167]:
#heatmap visualization of global city humidity (%) reported in OpenWeatherMap

fig = gmaps.figure()
layer_humidity = gmaps.heatmap_layer(dat[["Lat", "Lng"]], 
                                     weights= dat["Humidity (%)"], 
                                     dissipating=False, max_intensity=100,
                                     point_radius=1.5)
fig.add_layer(layer_humidity)
fig

Figure(layout=FigureLayout(height='420px'))

##### Top Cities by Weather Conditions

In [138]:
#Narrowing down df to identify top cities based on weather conditions.
dat.head()
selected_cities = dat.loc[(dat["Humidity (%)"] < 15) &  #low humidity
                          (dat["Cloudiness (%)"] < 20) &  #low cloudiness
                          (dat["Temperature (F)"] < (29*(9/5)+32)) &  #cool temperature
                          (dat["Temperature (F)"] > (17*(9/5)+32)), :]
print(f'Global cities df narrowed down to {len(selected_cities)} cities')

Global cities df narrowed down to 15 cities


In [139]:
selected_cities["Selected Hotel"] = "NA"
selected_cities["Hotel Lat"] = "NA"
selected_cities["Hotel Lng"] = "NA"

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i in range(0, len(selected_cities)):
    
    params = {
    "location": f'{selected_cities.iloc[i,2]}, {selected_cities.iloc[i,3]}',
    "keyword": "Hotel",
    "radius": 5000,
    "key": gkey
    }
    response = requests.get(url, params=params).json()
    
    try:
        print(f"{i}: Analyzing hotels in city: {selected_cities.iloc[i,0]}")
        
        selected_cities.iloc[i,10] = response["results"][0]["name"]
        selected_cities.iloc[i,11] = response["results"][0]["geometry"]["location"]["lat"]
        selected_cities.iloc[i,12] = response["results"][0]["geometry"]["location"]["lng"]

    except: 
        print(f"(Warning!) No hotel found at 5km radius of city: {selected_cities.iloc[i,0]}")
        pass
    
    print("--------------------------")

selected_cities.head()

0: Analyzing hotels in city: saint george
--------------------------
1: Analyzing hotels in city: shache
--------------------------
2: Analyzing hotels in city: sakakah
--------------------------
3: Analyzing hotels in city: tahta
--------------------------
4: Analyzing hotels in city: cortez
--------------------------
5: Analyzing hotels in city: nalut
(Warning!) No hotel found at 5km radius of city: nalut
--------------------------
6: Analyzing hotels in city: cedar city
--------------------------
7: Analyzing hotels in city: dehloran
--------------------------
8: Analyzing hotels in city: diffa
--------------------------
9: Analyzing hotels in city: saucillo
(Warning!) No hotel found at 5km radius of city: saucillo
--------------------------
10: Analyzing hotels in city: pandaria
--------------------------
11: Analyzing hotels in city: kingman
--------------------------
12: Analyzing hotels in city: winnemucca
--------------------------
13: Analyzing hotels in city: sohag
----------

,City,Country_code,Lat,Lng,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Preassure,Sea Level (ft),Selected Hotel,Hotel Lat,Hotel Lng
89,saint george,bm,37.1041,-113.5841,82.96,12.0,1.0,8.05,1024.0,NaN,SpringHill Suites by Marriott St. George Washi...,37.132784,-113.525534
418,shache,cn,38.4167,77.2406,66.76,11.0,1.0,6.71,1009.0,1009.0,Xinsheng Hotel,38.414222,77.247519
593,sakakah,sa,29.9697,40.2064,75.20,14.0,0.0,3.44,1018.0,NaN,Raoum Inn Hotel,29.945274,40.181156
788,tahta,eg,26.7693,31.5021,78.80,11.0,0.0,6.91,1011.0,NaN,El Salam Hotel,26.768816,31.504883
1017,cortez,us,37.3489,-108.5859,72.95,11.0,1.0,19.57,1026.0,NaN,Holiday Inn Express Mesa Verde-Cortez,37.348616,-108.558436


In [171]:
#City humidty heatmap compared with hotels in top selected cities

#dropping cities where we did not find hotels
selected_cities = selected_cities.loc[selected_cities["Hotel Lat"] != "NA",:]

#building text content
textbox = []
for i in range(0, len(selected_cities)):
    textbox.append(f'<d1><dt>Hotel:</dt><dd>{selected_cities.iloc[i,10]}</dd><dt>City:</dt><dd>{selected_cities.iloc[i,0]}</dd><dt>Country:</dt><dd>{selected_cities.iloc[i,1]}</dd></d1>')
textbox

#plotting
layer_hotels = gmaps.marker_layer(selected_cities[["Hotel Lat", "Hotel Lng"]],
                                 info_box_content= textbox)                      
fig.add_layer(layer_hotels)
fig

Figure(layout=FigureLayout(height='420px'))

Very curious.... I would have never imagined going on vacations to any of those regions!!! Worth giving them a shot!!!